In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [2]:
from os.path import exists

<a id="anchor_metafields"></a>
# Meta-fields

<a id="anchor_metafields_dataseta"></a>
## [DatasetA](#anchor_dataseta) fields

In [6]:
### The path to the full dataset with all relevant fields
DATASETA_PATH = "data-compas/compas-scores-two-years.csv"

### The decision source to explain, needs to be 0/1 split
DATASETA_DECISION = "2YCOMPAS" # Original dataset "low-risk/high-risk" threshold split
# DATASETA_DECISION = "RACIST" # Explicitly judging by race group only, [TODO] with the same accuracy vs. same low/high risk split as original?

# [TODO] remove the below
# ### The dataset to use in this analysis
# DATASET_USED = '2YCOMPAS'
# # DATASET_USED = 'BSCOMPAS'
# # DATASET_USED = 'RDCOMPAS'

### Name of column to use in justification
DATASETA_DECISION_COLNAME = "ncol_decision"

<a id="anchor_metafields_datasetb"></a>
## [DatasetB](#anchor_datasetb) fields

<a id="anchor_metafields_explain"></a>
## [Explain](#anchor_explain) fields

<a id="anchor_metafields_evaluate"></a>
## [Evaluate](#anchor_evaluate) fields

In [3]:
# ### Fields accessible to use in rationalization?
# ### ... some of which are VERY unfair and/or illegal
# ACCESSIBLE_FIELDS = [
#     # 'sex', # >:(
#     # 'race', # >:(
#     # 'age', # >:(
#     # 'age_cat',
#     'juv_fel_count',
#     'juv_misd_count',
#     'juv_other_count',
#     'priors_count',
#     'c_charge_degree',
#     'c_charge_desc',
# ]

# ### Field that is the source of what we are rationalizing based on
# TRUERESULT_FIELD = 'two_year_recid'
# # TRUERESULT_FIELD = 'is_recid'
# ### Field we are trying to rationalize why it could be 1 vs 0
# # JUSTIFYING_FIELD = 'is_recid' # not going to use 'is_violent_recid' much here
# JUSTIFYING_FIELD = 'two_year_recid'
# ### Field that is 1 if justifying==trueresult, otherwise 0.
# ### This is DEFINED BY ME AND NOT IN THE ORIGINAL DATA
# WASCORRECT_FIELD = 'pred_accurate'

# ### Threshold for confidence range
# CONF_ALPHA = 0.05

# ### Seed to begin with in 'train'/'test' split sampling
# SAMPLING_SEED = 1
# ### Ratio of 'train'/'test' split sampling
# SPLIT_RATIO = 0.8
# ### Number of runs to try (increments seed by 1 repeatedly)
# NUM_RUNS = 5
# ### Maximum number of factors to include in the whole-dataset evaluation
# MAX_RATEXPL_FACTORS = 2

<a id="anchor_dataseta"></a>
# DatasetA: what decisions were made?

[Relevant metafields](#anchor_metafields_dataseta)

In [7]:
# see: https://github.com/propublica/compas-analysis/blob/master/Compas%20Analysis.ipynb
def dataseta_compas_filter(df_raw):
    df = df_raw
    df = df[df["days_b_screening_arrest"] >= -30]
    df = df[df["days_b_screening_arrest"] <= 30]
    df = df[df["is_recid"] != -1]
    df = df[df["c_charge_degree"] != "O"]
    df = df[df["score_text"] != "N/A"]
    return df

In [ ]:
def dataseta_decision_compas(df_filtered):
    return df_filtered["is_recid"]

In [15]:
def dataseta_decision_racist(df_filtered):
    # Match for the number of positive predictions
    num_pos_pred = sum(df_filtered["is_recid"].values)
    # sort by race, apply positives from 1->0 in alphabetic race order
    df_filtered = df_filtered.sort_values("race")
    df_filtered["temp"] = [
        (1 if i<num_pos_pred else 0) 
        for i in range(len(df_filtered))
    ]
    # re-order to match original dataset index
    df_filtered = df_filtered.sort_index()
    return df_filtered["temp"]

In [16]:
df_raw = pd.read_csv(DATASETA_PATH)
df_filtered = dataseta_compas_filter(df_raw)
print(dataseta_decision_racist(df_filtered))

0       0
1       1
2       1
5       0
6       0
7       0
8       0
10      0
11      1
12      0
13      1
14      0
15      1
16      0
17      1
18      0
19      0
20      1
21      1
22      0
23      0
24      0
25      0
26      0
27      1
29      1
30      1
31      0
32      1
33      0
34      0
36      0
37      0
38      1
39      0
40      1
41      1
42      0
43      1
44      1
45      0
46      1
47      0
49      1
50      0
51      0
54      1
55      1
56      0
57      1
58      0
59      1
61      1
62      0
63      1
65      1
66      0
67      1
68      0
69      1
70      0
71      0
72      1
73      1
74      0
75      1
76      1
77      0
78      1
80      1
81      1
82      1
83      0
84      1
85      0
86      1
88      1
89      1
90      0
91      0
92      0
94      0
95      0
97      0
98      0
99      1
101     1
102     0
103     0
104     0
105     0
106     0
108     0
109     1
110     0
111     0
112     1
113     1
114     0
115     1


<a id="anchor_datasetb"></a>
# DatasetB: what explanations could be used?

[Relevant metafields](#anchor_metafields_datasetb)

<a id="anchor_explain"></a>
# Explain: what explanations were actually used for each case?

[Relevant metafields](#anchor_metafields_explain)

<a id="anchor_evaluate"></a>
# Evaluate: what are the faithfulness metrics for a given set of used explanations?

[Relevant metafields](anchor_metafields_evaluate)